In [19]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import image_select

from sklearn.ensemble import RandomForestClassifier
from time import perf_counter
from sklearn import tree

import time

In [7]:
root = Tk()
filename = askopenfilename()
root.destroy()

In [9]:
plant, back = image_select.get_training(filename)
X_train, y_train = image_select.prepare_training(plant, back)

Obtaining plant parts...
Obtaining background parts...


In [11]:
def train_plants(file):
    plant, back = image_select.get_training(file)
    X_train, y_train = image_select.prepare_training(plant, back)
    
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X_train, y_train)
    
    return clf

In [25]:
tree = train_plants(filename)

Obtaining plant parts...
Obtaining background parts...


In [26]:
def remove_back(image, classifier):
    start = time.perf_counter()
    
    img_BGR = cv2.imread(image)
    img_HSV = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
    img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB)
    img_final = np.concatenate((img_BGR, img_HSV, img_LAB), axis=2).reshape((-1, 9))
    
    test_pred = classifier.predict(img_final)
    test_pred = test_pred.reshape((len(test_pred),1))
    test_pred = np.concatenate((test_pred, test_pred, test_pred), axis=1).reshape((img_BGR.shape[0], img_BGR.shape[1], 3))
    
    new_image = np.multiply(img_BGR, test_pred).astype(np.uint8)
    
    end = time.perf_counter()
    print(end - start)
    
    cv2.imshow('trained image', new_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    return new_image

In [27]:
new_image = remove_back(filename, tree)

4.082037141561727
